In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext Cython

import time
import decimal
import numpy as np
import warnings
import pandas as pd
from pathlib import Path
import sys
sys.path.append(str(Path.home()) + '/rpi-canary-eval/modules')

import data_process, data_functions, diabetes_onset
from path import path_dict

# 1. Load All the final datasets into program.
Processed Datasets are processed PMPM, processed Canary and Biometrics.

In [2]:
canary, biometrics, pmpm = data_process.load_data(path_dict)

In [3]:
canary = canary.dropna()

In [4]:
pmpm.head()

PERS_ID     MYR MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  \
0  2564600  201710        F               99        176       0.0       0.0   
1  2564600  202001        F              102          0       0.0       0.0   
2  2564600  202005        F              102          0       0.0       0.0   
3  2564600  201704        F               99          0       0.0       0.0   
4  2564600  201804        F              100         12       0.0       0.0   

   ELIXHAUSER  ELX_GRP_1  ELX_GRP_10  ...  ELX_GRP_29  ELX_GRP_3  ELX_GRP_30  \
0         3.0        0.0         0.0  ...         0.0        0.0         0.0   
1         0.0        0.0         0.0  ...         0.0        0.0         0.0   
2         0.0        0.0         0.0  ...         0.0        0.0         0.0   
3         0.0        0.0         0.0  ...         0.0        0.0         0.0   
4         0.0        0.0         0.0  ...         0.0        0.0         0.0   

   ELX_GRP_31  ELX_GRP_4  ELX_GRP_5  ELX_GRP_6  ELX_GRP_7  ELX_GRP_8  \
0         0.0        0.0        1.0        1.0        0.0        0.0   
1         0.0        0.0        0.0        0.0        0.0        0.0   
2         0.0        0.0        0.0        0.0        0.0        0.0   
3         0.0        0.0        0.0        0.0        0.0        0.0   
4         0.0        0.0        0.0        0.0        0.0        0.0   

   ELX_GRP_9  
0        1.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0  

[5 rows x 38 columns]

**Temporary PMPM Processing** [Skip this step if you are not using ELX codes instead of CC Codes]

Weights collected from - https://cran.r-project.org/web/packages/comorbidity/vignettes/comorbidityscores.html <br/>
PMPM doesn't have data for ELX GRP 17 - AIDS/HIV, but the weight for that is 0, so that's handled anyway. Removed that from weight list. So now = 30 weights <br/>
Score range can be -19 to +89 (from paper). In this study, the range is -10 to 48

In [5]:
pmpm = data_process.elx_pmpm_processing(pmpm.copy())

In [6]:
pmpm.head()

PERS_ID     MYR MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  \
0  2564600  201710        F               99        176       0.0       0.0   
1  2564600  202001        F              102          0       0.0       0.0   
2  2564600  202005        F              102          0       0.0       0.0   
3  2564600  201704        F               99          0       0.0       0.0   
4  2564600  201804        F              100         12       0.0       0.0   

   ELIXHAUSER  ELX_GRP_1  ELX_GRP_2  ...  ELX_GRP_23  ELX_GRP_24  ELX_GRP_25  \
0         3.0        0.0        0.0  ...         0.0         0.0         0.0   
1         0.0        0.0        0.0  ...         0.0         0.0         0.0   
2         0.0        0.0        0.0  ...         0.0         0.0         0.0   
3         0.0        0.0        0.0  ...         0.0         0.0         0.0   
4         0.0        0.0        0.0  ...         0.0         0.0         0.0   

   ELX_GRP_26  ELX_GRP_27  ELX_GRP_28  ELX_GRP_29  ELX_GRP_30  ELX_GRP_31  \
0         0.0         0.0         0.0         0.0         0.0         0.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         0.0         0.0         0.0         0.0         0.0   
3         0.0         0.0         0.0         0.0         0.0         0.0   
4         0.0         0.0         0.0         0.0         0.0         0.0   

   ELX_SCORE  
0        8.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0  

[5 rows x 39 columns]

# 2. Generate data by changing outcome and other parameters

**Step 2.1 Prepare Data file for any custome outcome here (Optional):** Generates outcome for diabetes onset (using PMPM), by generating a 'biometrics like' file for diabetes onset. Only need to do this if we are using outcomes outside of Biometrics file. <br/>
For diabetes onset, bigger_better=False, Outcome 1 is good, 0 is bad, 0 means x number of people transitioned from non-diabetes to diabetes (diabetes onset). <br/>
Run only if you want diabetes onset as outcome, otherwise comment this line

In [7]:
%%time
outcome_col_in_pmpm = "ELX_GRP_12"
biometrics = data_process.gen_diab_onset(pmpm, outcome_col_in_pmpm)

CPU times: user 25.8 s, sys: 696 ms, total: 26.5 s
Wall time: 26.5 s


**Step 2.2 Generate Final Data:** Generate the final dataset with features, outcomes, and treatment column. Change the outcome_name and other parameters as necessary.

In [8]:
%%time
th = 0.5
merge_how = 'inner'
control_filters = 'new'
outcome_filename = "DIAB_ONSET"
warnings.filterwarnings("ignore", category=FutureWarning)
df, treated_help =  data_functions.create_causal_data(canary.copy(), pmpm.copy(), biometrics.copy(), 
                      outcome_under_study = outcome_col_in_pmpm, drop_single = True,
                      enroll_filt = True, enroll_months = 12, merge_how = merge_how,
                      max_days_apart = 365, min_days_apart = 365, threshold = th,
                                       bigger_better = False, window = 3,
                                      control_filters = control_filters)

Starting Control Outcomes New 0.33s per 100 id processing
Total of: 51358 ids, time to finish: 154.074 s


/home/nneehal/rpi-canary-eval/modules/data_functions.py:267: RuntimeWarning: invalid value encountered in double_scalars
  data['OUTCOME2']  = (data['RESULT_VALUE'].iloc[-1] - data['RESULT_VALUE'].iloc[0])/data['RESULT_VALUE'].iloc[0]


CPU times: user 3min 59s, sys: 5.02 s, total: 4min 4s
Wall time: 4min 4s


# 3. Save final data file
Save the final data file. It will be used for input to DoWhy Causal Inference package. 

In [9]:
df.columns

Index(['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'ALLOW_ER', 'ALLOW_IP',
       'BIOMETRIC_CATEGORY', 'ELIXHAUSER', 'ELX_GRP_1', 'ELX_GRP_10',
       'ELX_GRP_11', 'ELX_GRP_12', 'ELX_GRP_13', 'ELX_GRP_14', 'ELX_GRP_15',
       'ELX_GRP_16', 'ELX_GRP_18', 'ELX_GRP_19', 'ELX_GRP_2', 'ELX_GRP_20',
       'ELX_GRP_21', 'ELX_GRP_22', 'ELX_GRP_23', 'ELX_GRP_24', 'ELX_GRP_25',
       'ELX_GRP_26', 'ELX_GRP_27', 'ELX_GRP_28', 'ELX_GRP_29', 'ELX_GRP_3',
       'ELX_GRP_30', 'ELX_GRP_31', 'ELX_GRP_4', 'ELX_GRP_5', 'ELX_GRP_6',
       'ELX_GRP_7', 'ELX_GRP_8', 'ELX_GRP_9', 'ELX_SCORE', 'MBR_GNDR',
       'OUTCOME', 'OUTCOME2', 'OUTCOME3', 'PERS_ID', 'RESULT_VALUE',
       'RESULT_VALUE2', 'TREATED'],
      dtype='object')

In [10]:
%%time
df.to_pickle(path_dict['generated_causal_data_path'])

CPU times: user 247 ms, sys: 280 ms, total: 527 ms
Wall time: 526 ms
